# Analyse differentielle de midori

In [1]:
from midori64 import *
import midori64_c
from tqdm.notebook import trange, tqdm
import random
from termcolor import colored
import math

## Utilities

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

Determinism is good

In [3]:
random.seed(0xdead)

Utility for drawing heatmaps

In [4]:
def ascii_heatmap(data: list[list[int]], threshold_1=None, threshold_2=None, threshold_3=None) -> str:
    ret = ''

    for row in data:
        for el in row:
            c = '░'
            if threshold_1 is not None and threshold_1(el):
                c = '▒'
            if threshold_2 is not None and threshold_2(el):
                c = '▓'
            if threshold_3 is not None and threshold_3(el):
                c = '█'
            ret += c + ' '
        ret += '\n'

    return ret

## Question 1

Calculer la DDT de Midori

In [5]:
DDT = [[0 for _ in range(16)] for _ in range(16)]

for x in range(16):
    for y in range(16):
        DDT[x ^ y][S_BOX[x] ^ S_BOX[y]] += 1

DDT

[[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 4, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0],
 [0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 4, 2, 2, 2, 0, 0, 0, 2, 0, 2],
 [0, 2, 4, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0],
 [0, 2, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 0, 0, 0],
 [0, 2, 0, 4, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 4, 2, 0],
 [0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0],
 [0, 0, 4, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4],
 [0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 4, 0, 2, 0, 2],
 [0, 0, 4, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0],
 [0, 0, 0, 2, 0, 0, 2, 4, 0, 0, 4, 2, 0, 0, 2, 0],
 [0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 4, 2],
 [0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 4, 2, 0, 0, 2, 4]]

Here's a more visual representation

In [6]:
print(ascii_heatmap(
    DDT,
    threshold_1=lambda x: x == 2,
    threshold_3=lambda x: x >= 4,
))

print('Legend\n\t- ░ >= 0\n\t- ▒ >= 2\n\t- █ >= 4')

█ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ▒ █ ░ ▒ ▒ ▒ ░ ▒ ░ ░ ░ ░ ░ ▒ ░ 
░ █ ░ ░ █ ░ ░ ░ ░ █ ░ ░ █ ░ ░ ░ 
░ ░ ░ ░ ▒ ░ █ ▒ ▒ ▒ ░ ░ ░ ▒ ░ ▒ 
░ ▒ █ ▒ ▒ ▒ ░ ░ ▒ ░ ░ ▒ ░ ░ ░ ░ 
░ ▒ ░ ░ ▒ ░ ░ █ ░ ▒ █ ░ ▒ ░ ░ ░ 
░ ▒ ░ █ ░ ░ ░ ▒ ▒ ░ ░ ░ ▒ ▒ ░ ▒ 
░ ░ ░ ▒ ░ █ ▒ ░ ░ ░ ░ ▒ ░ █ ▒ ░ 
░ ▒ ░ ▒ ▒ ░ ▒ ░ ░ ▒ ░ ▒ ▒ ░ ▒ ░ 
░ ░ █ ▒ ░ ▒ ░ ░ ▒ ▒ ░ ▒ ▒ ░ ░ ░ 
░ ░ ░ ░ ░ █ ░ ░ ░ ░ █ ░ ░ █ ░ █ 
░ ░ ░ ░ ▒ ░ ░ ▒ ▒ ▒ ░ █ ░ ▒ ░ ▒ 
░ ░ █ ░ ░ ▒ ▒ ░ ▒ ▒ ░ ░ ▒ ░ ▒ ░ 
░ ░ ░ ▒ ░ ░ ▒ █ ░ ░ █ ▒ ░ ░ ▒ ░ 
░ ▒ ░ ░ ░ ░ ░ ▒ ▒ ░ ░ ░ ▒ ▒ █ ▒ 
░ ░ ░ ▒ ░ ░ ▒ ░ ░ ░ █ ▒ ░ ░ ▒ █ 

Legend
	- ░ >= 0
	- ▒ >= 2
	- █ >= 4


## Question 2

Trouver la meilleur differentiel pour ce chemin

![](./images/png/midori_square_attack5_dark.drawio.png)

Let's consider what happens in the `mixColumn`

$$

\begin{pmatrix}
0 & 1 & 1 & 1 \\
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 1 \\
1 & 1 & 1 & 0 \\
\end{pmatrix}
\begin{pmatrix}
0 \\
0 \\
\delta_1 \\
\delta_2 \\
\end{pmatrix}
=
\begin{pmatrix}
0 \\
0 \\
\delta_2 \\
\delta_1 \\
\end{pmatrix}
$$

To preserve the pattern, we want $\delta_1 = \delta_2$ at the entry of `mixColumns`.

<!-- Because of the key, we want $SB(x) \oplus k_3 = SB(y) \oplus k_4$ -->

All our states share the same none null values, we note this value $\delta_i$ in state $\Delta_i$. The area of this difference is colored in white in the graph.

We need to find a _likely_ path where the differences going into `mixColumns`. By looking into our DDT, we notice that the column 2 has many 4 and many zeros. If we fix $\delta_1$ to 2, using the DDT we can calculate $\delta_2$:
- 1 with probability $\left(\frac{1}{4}\right)^2 = 0.0625$, we then get $1 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6= 2$ with probability $\left(\frac{1}{4}\right)^2 = 0.0625$
- 4 with probability $0.0625$ we then get $4 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$
- 9 with probability $0.0625$ we then get $9 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$
- 12 with probability $0.0625$ we then get $12 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$

This gives us a final probability of
$$
P(\Delta_6) = 4 \times \left(\frac{1}{4}\right)^4 = 1.6\%
$$

## Question 3


Evaluer experimentalement cette probabilité

Here's our modified round with an extra `subCell` at the end

In [7]:
def modified_round(msg: list[int], key: list[int]) -> list[int]:
    ct = subCell(msg)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, key)
    ct = subCell(ct)
    return ct

The pattern we are interested in is
$$
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\delta_\text{out} & 0 & 0 & 0 \\
\delta_\text{out} & 0 & 0 & 0 \\
\end{pmatrix}
$$

In [8]:
def has_pattern(ct1, ct2):
    ret = True
    for i in range(16):
        match i:
            case 2:
                ret = ret and (ct1[2] ^ ct2[2] == ct1[3] ^ ct2[3])
            case 3:
                pass
            case _:
                ret = ret and ct1[i] == ct2[i]
    return ret

In [9]:
ITERATIONS = 10_000

score = [[0 for _ in range(16)] for _ in range(16)]

for delta in trange(1, 16):
    for _ in range(ITERATIONS):
        key = [random.randint(0, 15) for _ in range(16)]
        m1 = [random.randint(0, 15) for _ in range(16)]
        m2 = [x for x in m1]
        m2[5] ^= delta
        m2[15] ^= delta


        ct1 = modified_round(m1, key)
        ct2 = modified_round(m2, key)

        if has_pattern(ct1, ct2):
            score[delta][ct1[2] ^ ct2[2]] += 1

  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
m = max([score[y][x] for x in range(16) for y in range(16)])
probabilities_string = ''

for y in range(len(score)):
    for x in range(len(score[y])):
        if score[y][x] >= 0.8 * m:
            probabilities_string += f'\n\t- delta_in={y} and delta_out={x} probability={score[y][x] / ITERATIONS * 100:.2f}%'


print(ascii_heatmap(score, threshold_3=lambda x: x >= 0.8 * m))
print(f'Found probabilities:{probabilities_string}')

░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ █ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ █ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 

Found probabilities:
	- delta_in=2 and delta_out=2 probability=1.42%
	- delta_in=10 and delta_out=10 probability=1.43%


We find a value close to the expected probability of $1.6\%$

## Question 4

In [11]:
random.seed(0xdeadbeef)

keys = [
    [random.randint(0, 15) for _ in range(16)]
    for _ in range(10)
]

Attack!

![](./images/png/midori_square_attack6_dark.drawio.png)

We first need to retrieve couples $(P, C)$. We can optimize our code here by storing these couples in a hash table indexed by the inactive nibbles of $C$ and containing as values the active nibbles of $P$

In [35]:
def enc(m, ks):
    ct = addRoundKey(m, ks[0])

    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[1])
    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[2])
    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[3])
    ct = subCell(ct)
    ct = addRoundKey(ct , ks[4])
    return ct

In [40]:
for _ in range(1000):
    m = [i for i in range(16)]
    ks = [[random.randint(0, 15) for _ in range(16)] for _ in range(10)]
    assert enc(m, ks) == midori64_c.encrypt_differential(m, ks)

In [27]:
m0 = [0 for _ in range(16)]
m0[1] = 2
m0[2] = 2
m0[7] = 2
m0[11] = 2
m0[13] = 2
m0[14] = 2

mixColumns(shuffleCell(m0))

[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]

In [12]:
hash_table: dict[str, list[int]] = {}

In [13]:
def has_pattern_differential(ct: list[int]) -> bool:
    ret = True
    for i in range(16):
        if i in [8, 10, 11, 12, 13, 15]:
            ret = ret and ct[i] == 2
        else:
            ret = ret and ct[i] == 0

In [14]:
def get_key(ct: list[int]) -> str:
    l = []
    for i in [0, 1, 2, 3, 4, 5, 6, 7, 9, 14]:
        l.append(ct[i])
    return str(l)

We can fix a message and modify only certain active nibbles (see figure).

In [15]:
msg = [random.randint(0, 15) for _ in range(16)]

In [16]:
couples = []

In [50]:
hash_table: dict[str, list[int]] = {}

nibbles = [(n1, n2, n3, n4, n5, n6)
           for n1 in range(16)
           for n2 in range(16)
           for n3 in range(16)
           for n4 in range(16)
           for n5 in range(16)
           for n6 in range(16)
           ]

for (n1, n2, n3, n4, n5, n6) in tqdm(nibbles):
    m = [x for x in msg]

    m[1] = n1
    m[2] = n2
    m[7] = n3
    m[11] = n4
    m[13] = n5
    m[14] = n6

    midori64_c.encrypt_differential(m, keys)

    ct1 = m[8]
    ct2 = m[10]
    ct3 = m[11]
    ct4 = m[12]
    ct5 = m[13]
    ct6 = m[15]

    # if has_pattern_differential(m):
    # couples.append([n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6])
    delta = get_key(m)
    if delta in hash_table:
        hash_table[delta].append([n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6])
    else:
        hash_table[delta] = [[n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6]]


  0%|          | 0/16777216 [00:00<?, ?it/s]

In [51]:
len(hash_table)

65536

In [52]:
def get_key_guess(n1: list[int], n2: list[int]) -> str:

    for i in range(16):
        if n1[i] == n2[i]:
            return None

    for k1 in range(16):
        if S_BOX[n1[0] ^ k1] ^ S_BOX[n2[0] ^ k1] != 2:
            continue

        for k2 in range(16):
            if S_BOX[n1[1] ^ k2] ^ S_BOX[n2[1] ^ k2] != 2:
                continue

            for k3 in range(16):
                if S_BOX[n1[2] ^ k3] ^ S_BOX[n2[2] ^ k3] != 2:
                    continue

                for k4 in range(16):
                    if S_BOX[n1[3] ^ k4] ^ S_BOX[n2[3] ^ k4] != 2:
                        continue

                    for k5 in range(16):
                        if S_BOX[n1[4] ^ k5] ^ S_BOX[n2[4] ^ k5] != 2:
                            continue

                        for k6 in range(16):
                            if S_BOX[n1[5] ^ k6] ^ S_BOX[n2[5] ^ k6] != 2:
                                continue

                            for k7 in range(16):
                                if S_BOX[n1[6] ^ k7] ^ S_BOX[n2[6] ^ k7] != 2:
                                    continue

                                for k8 in range(16):
                                    if S_BOX[n1[7] ^ k8] ^ S_BOX[n2[7] ^ k8] != 2:
                                        continue

                                    for k9 in range(16):
                                        if S_BOX[n1[8] ^ k9] ^ S_BOX[n2[8] ^ k9] != 2:
                                            continue

                                        for k10 in range(16):
                                            if S_BOX[n1[9] ^ k10] ^ S_BOX[n2[9] ^ k10] != 2:
                                                continue

                                            for k11 in range(16):
                                                if S_BOX[n1[10] ^ k11] ^ S_BOX[n2[10] ^ k11] != 2:
                                                    continue

                                                for k12 in range(16):
                                                    if S_BOX[n1[11] ^ k12] ^ S_BOX[n2[11] ^ k12] != 2:
                                                        continue
                                                    return [str([k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12])]

    return None

In [53]:
key_guesses = {}

for nibbles in tqdm(hash_table):
    for n1 in hash_table[nibbles]:
        for n2 in hash_table[nibbles]:
            if n1 == n2:
                continue

            key_guess = get_key_guess(n1, n2)

            if key_guess is not None:
                print("GUESSED")
                if key_guess in key_guesses:
                    key_guesses[key_guess] += 1
                else:
                    key_guesses[key_guess] = 1

  0%|          | 0/65536 [00:00<?, ?it/s]

In [ ]:
hash_table

0

In [25]:
max([(k, hash_table[k]) for k in hash_table], key=lambda x: x[1])

('[11, 15, 15, 4, 15, 5, 9, 15, 7, 9]',
 [[15, 15, 15, 1, 1, 9, 15, 6, 1, 3, 1, 3],
  [15, 15, 15, 2, 1, 9, 15, 6, 2, 3, 1, 3],
  [15, 15, 15, 2, 3, 9, 15, 6, 2, 3, 3, 3],
  [15, 15, 15, 2, 7, 9, 15, 6, 2, 3, 7, 3],
  [15, 15, 15, 4, 5, 9, 15, 6, 4, 3, 5, 3],
  [15, 15, 15, 4, 7, 9, 15, 6, 4, 3, 7, 3],
  [15, 15, 15, 4, 8, 9, 15, 6, 4, 3, 8, 3],
  [15, 15, 15, 4, 15, 9, 15, 6, 4, 3, 15, 3],
  [15, 15, 15, 5, 1, 9, 15, 6, 5, 3, 1, 3],
  [15, 15, 15, 5, 2, 9, 15, 6, 5, 3, 2, 3],
  [15, 15, 15, 5, 4, 9, 15, 6, 5, 3, 4, 3],
  [15, 15, 15, 5, 9, 9, 15, 6, 5, 3, 9, 3],
  [15, 15, 15, 5, 10, 9, 15, 6, 5, 3, 10, 3],
  [15, 15, 15, 5, 11, 9, 15, 6, 5, 3, 11, 3],
  [15, 15, 15, 6, 2, 9, 15, 6, 6, 3, 2, 3],
  [15, 15, 15, 6, 11, 9, 15, 6, 6, 3, 11, 3],
  [15, 15, 15, 6, 13, 9, 15, 6, 6, 3, 13, 3],
  [15, 15, 15, 7, 9, 9, 15, 6, 7, 3, 9, 3],
  [15, 15, 15, 7, 12, 9, 15, 6, 7, 3, 12, 3],
  [15, 15, 15, 8, 14, 9, 15, 6, 8, 3, 14, 3],
  [15, 15, 15, 9, 3, 9, 15, 6, 9, 3, 3, 3],
  [15, 15, 15, 9, 8, 9

In [63]:
nibbles = '[11, 15, 15, 4, 15, 5, 9, 15, 7, 9]'
for n1 in hash_table[nibbles]:
        for n2 in hash_table[nibbles]:
            for i in range(1):
                if n1[i] == n2[i]:
                    break
            else:
                print("FOUND", n1, n2)

In [ ]:
list(hash_table)[:10]

['[11, 0, 0, 4, 15, 5, 9, 0, 7, 0]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 2]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 15]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 6]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 7]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 13]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 1]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 3]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 4]',
 '[11, 0, 0, 4, 15, 5, 9, 0, 7, 8]']

In [24]:
hash_table['[11, 0, 0, 4, 15, 5, 9, 0, 7, 0]'][::-1]

[[0, 0, 0, 15, 15, 0, 15, 6, 15, 3, 15, 3],
 [0, 0, 0, 15, 14, 0, 15, 6, 15, 3, 14, 3],
 [0, 0, 0, 15, 8, 0, 15, 6, 15, 3, 8, 3],
 [0, 0, 0, 15, 4, 0, 15, 6, 15, 3, 4, 3],
 [0, 0, 0, 14, 10, 0, 15, 6, 14, 3, 10, 3],
 [0, 0, 0, 14, 7, 0, 15, 6, 14, 3, 7, 3],
 [0, 0, 0, 14, 0, 0, 15, 6, 14, 3, 0, 3],
 [0, 0, 0, 13, 8, 0, 15, 6, 13, 3, 8, 3],
 [0, 0, 0, 13, 4, 0, 15, 6, 13, 3, 4, 3],
 [0, 0, 0, 13, 0, 0, 15, 6, 13, 3, 0, 3],
 [0, 0, 0, 12, 15, 0, 15, 6, 12, 3, 15, 3],
 [0, 0, 0, 12, 9, 0, 15, 6, 12, 3, 9, 3],
 [0, 0, 0, 12, 7, 0, 15, 6, 12, 3, 7, 3],
 [0, 0, 0, 12, 5, 0, 15, 6, 12, 3, 5, 3],
 [0, 0, 0, 11, 3, 0, 15, 6, 11, 3, 3, 3],
 [0, 0, 0, 10, 7, 0, 15, 6, 10, 3, 7, 3],
 [0, 0, 0, 10, 1, 0, 15, 6, 10, 3, 1, 3],
 [0, 0, 0, 9, 4, 0, 15, 6, 9, 3, 4, 3],
 [0, 0, 0, 8, 10, 0, 15, 6, 8, 3, 10, 3],
 [0, 0, 0, 8, 2, 0, 15, 6, 8, 3, 2, 3],
 [0, 0, 0, 7, 0, 0, 15, 6, 7, 3, 0, 3],
 [0, 0, 0, 6, 7, 0, 15, 6, 6, 3, 7, 3],
 [0, 0, 0, 6, 5, 0, 15, 6, 6, 3, 5, 3],
 [0, 0, 0, 5, 12, 0, 15, 6, 5, 3, 12